In [ ]:
!pip install langchain_openai
!pip install langchain_core
!pip install langchain_community
!pip install langserve
!pip install pineconedb
!pip install openai pinecone-client langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49

In [ ]:
import pandas as pd
import langchain_openai as lo

In [ ]:
a='/content/drive/MyDrive/dat.csv'
docs=pd.read_csv(a)
docs.head()

,Unnamed: 0,Age,Gender,Symptoms,Disease
0,0,NA,NA,"pain in the upper right abdomen, severe pain,...",Possible gallbladder disease (due to family h...
1,1,30,Female,"Burning sensation during urination, increased...",Suspected bladder infection (urinary tract in...
2,2,NA,NA,"Intense pain in right shoulder, inability to ...",Suspected shoulder dislocation
3,3,NA,NA,"back pain, lower back pain, sharp pain, stiff...",mechanical back pain
4,4,5,Female,"Limping, Pain in upper right leg and hip, Dif...","Transient synovitis, Septic arthritis (to be ..."


In [ ]:
import os
os.environ['OPENAI_API_KEY']="sk-proj-0EqhXoapgpXdp5qY4T8cT3BlbkFJuXZEmImJThxwaj4K1paz"


In [ ]:
import langchain_community.vectorstores as vectorstores

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="ac11b5f4-e84e-4b91-87ef-97404b8db2e1")


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embed = OpenAIEmbeddings(
    model='text-embedding-ada-002',
    openai_api_key="sk-proj-0EqhXoapgpXdp5qY4T8cT3BlbkFJuXZEmImJThxwaj4K1paz"
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
docs=docs.dropna(subset=['Disease'])
docs.drop(docs[docs['Disease']==' NA'].index,inplace=True)
from collections import Counter
Counter(docs['Disease'])
docs['Symp'] = docs['Symptoms'].apply(lambda x: embed.embed_query(x))
docs.head()

<ipython-input-11-6ea46430a98e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docs.drop(docs[docs['Disease']==' NA'].index,inplace=True)


,Unnamed: 0,Age,Gender,Symptoms,Disease,Symp
0,0,NA,NA,"pain in the upper right abdomen, severe pain,...",Possible gallbladder disease (due to family h...,"[-0.00393627356914304, -0.022464176968886102, ..."
1,1,30,Female,"Burning sensation during urination, increased...",Suspected bladder infection (urinary tract in...,"[0.0045958826453449605, -0.008622309917254474,..."
2,2,NA,NA,"Intense pain in right shoulder, inability to ...",Suspected shoulder dislocation,"[-0.025765504022781178, 0.006403670480912936, ..."
3,3,NA,NA,"back pain, lower back pain, sharp pain, stiff...",mechanical back pain,"[0.003139527832401235, -0.013581458770952426, ..."
4,4,5,Female,"Limping, Pain in upper right leg and hip, Dif...","Transient synovitis, Septic arthritis (to be ...","[-0.006454581428605978, 1.362841619857281e-05,..."


In [ ]:
#docs.drop(columns=['Age','Gender'],inplace=True)
docs['Unnamed: 0']=docs['Unnamed: 0'].apply(lambda x: str(x))
docs.head()

,Unnamed: 0,Age,Gender,Symptoms,Disease,Symp
0,0,NA,NA,"pain in the upper right abdomen, severe pain,...",Possible gallbladder disease (due to family h...,"[-0.00393627356914304, -0.022464176968886102, ..."
1,1,30,Female,"Burning sensation during urination, increased...",Suspected bladder infection (urinary tract in...,"[0.0045958826453449605, -0.008622309917254474,..."
2,2,NA,NA,"Intense pain in right shoulder, inability to ...",Suspected shoulder dislocation,"[-0.025765504022781178, 0.006403670480912936, ..."
3,3,NA,NA,"back pain, lower back pain, sharp pain, stiff...",mechanical back pain,"[0.003139527832401235, -0.013581458770952426, ..."
4,4,5,Female,"Limping, Pain in upper right leg and hip, Dif...","Transient synovitis, Septic arthritis (to be ...","[-0.006454581428605978, 1.362841619857281e-05,..."


In [ ]:
pc.create_index(
    name="disease-symptoms-gpt-4",
    dimension=len(docs['Symp'][0]), # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '5bf2158db52106217fe03d75b239e73b', 'Date': 'Sat, 03 Aug 2024 22:28:13 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
index = pc.Index("disease-symptoms-gpt-4")
# wait a moment for connection

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 264}},
 'total_vector_count': 264}

In [ ]:
docs.head()
for i,j in docs.iterrows():
  index.upsert(vectors=[(j['Unnamed: 0'],j['Symp'],{'text':j['Symptoms'],'disease':j['Disease']})])
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 264}},
 'total_vector_count': 264}

In [ ]:
from langchain.vectorstores import Pinecone
# switch back to normal index for langchain

vectorstore = Pinecone(index, embed.embed_query, "text")


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 0.3.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


Semantic Search



In [ ]:
query = "pain in the upper right abdomen, severe pain, nausea, decreased energy, decreased appetite"


vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(metadata={'disease': ' Possible gallbladder disease (due to family history and symptoms)'}, page_content=' pain in the upper right abdomen, severe pain, nausea, decreased energy, decreased appetite'),
 Document(metadata={'disease': ' Possible gastrointestinal issue, further tests needed'}, page_content=' abdominal pain, crampy feeling, constipation, nausea, pain worsening over the last three days, pain localized near the right hip and below the belly button, occasional feverish feeling'),
 Document(metadata={'disease': ' Possible biliary colic or cholecystitis, further investigation needed'}, page_content=' Pain in the right side of the belly, aching in nature, started 8 hours ago, initially intermittent but constant for the past 5-6 hours, rated 8/10 in severity, worsened by eating, accompanied by nausea, no vomiting, no fever, no chills, no changes in bowel movements, no recent travel, slight fatigue, no changes in urinary patterns, no skin changes or rashes, no changes in 

In [ ]:
from langchain.vectorstores import Pinecone as LangChainPinecone
from langchain.prompts import PromptTemplate
from openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain_core.vectorstores import VectorStoreRetriever

prompt_template='''for the following symptoms, predict the disease:..
    Text:
    {context}'''
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context"]
)
# Create a RetrievalQA chain
retriever = VectorStoreRetriever(vectorstore=vectorstore)
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(api_key="sk-proj-0EqhXoapgpXdp5qY4T8cT3BlbkFJuXZEmImJThxwaj4K1paz",model_name='gpt-4o',
    temperature=0.0),
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": PROMPT},)

In [ ]:
q=' pain in the upper right abdomen, severe pain, nausea, decreased energy, decreased appetite'
answer = qa_chain.run(query=q)
print(answer)

Based on the symptoms provided, here are the potential diseases or conditions that could be associated with each set of symptoms:

1. **Pain in the upper right abdomen, severe pain, nausea, decreased energy, decreased appetite:**
   - **Potential Disease:** Gallstones or Cholecystitis (inflammation of the gallbladder). These symptoms are typical of gallbladder issues, especially the pain in the upper right abdomen and nausea.

2. **Abdominal pain, crampy feeling, constipation, nausea, pain worsening over the last three days, pain localized near the right hip and below the belly button, occasional feverish feeling:**
   - **Potential Disease:** Appendicitis. The localization of pain near the right hip and below the belly button, along with worsening pain and occasional fever, are classic signs of appendicitis.

3. **Pain in the right side of the belly, aching in nature, started 8 hours ago, initially intermittent but constant for the past 5-6 hours, rated 8/10 in severity, worsened by e

In [ ]:
q=' pain in the upper right abdomen'
answer = qa_chain.run(query=q)
print(answer)

Pain in the upper right abdomen can be caused by various conditions. Some potential causes include:

1. **Gallstones**: These can block the bile ducts, causing severe pain, nausea, and vomiting.
2. **Hepatitis**: Inflammation of the liver can cause pain in the upper right abdomen, along with symptoms like fatigue, jaundice, and loss of appetite.
3. **Liver abscess**: An infection in the liver can cause pain, fever, and other systemic symptoms.
4. **Peptic ulcer**: Ulcers in the stomach or the first part of the small intestine can cause upper abdominal pain, especially after eating.
5. **Cholecystitis**: Inflammation of the gallbladder, often due to gallstones, can cause severe pain, fever, and nausea.

Given the context provided, it is important to seek medical attention to determine the exact cause and receive appropriate treatment.


In [ ]:
import openai
from langchain.chains import LLMChain
openai.api_key = 'sk-proj-0EqhXoapgpXdp5qY4T8cT3BlbkFJuXZEmImJThxwaj4K1paz'
from langchain_openai import ChatOpenAI
# Initialize OpenAI LLM
llm = OpenAI(api_key=openai.api_key)
chat_model = ChatOpenAI(api_key=openai.api_key,model="gpt-4o",temperature=0)
# Define a simple prompt template
prompt_template = PromptTemplate(
    input_variables=["data"],
    template='''given the following symptoms:.. predict the disease:..
    Text:
    {data}''')


# Create an LLMChain with the LLM and the prompt template
chain = LLMChain(llm=chat_model, prompt=prompt_template)
q=' pain in the upper right abdomen, severe pain, nausea, decreased energy, decreased appetite'
answer = chain.run(q)
print(answer)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


The symptoms described—pain in the upper right abdomen, severe pain, nausea, decreased energy, and decreased appetite—could be indicative of several conditions, but one common disease that fits this profile is **cholecystitis** (inflammation of the gallbladder). 

Cholecystitis often presents with pain in the upper right abdomen, which can be severe and may radiate to the back or right shoulder. Nausea and vomiting are also common, along with a general feeling of malaise, decreased energy, and loss of appetite.

However, these symptoms can also be associated with other conditions such as:

1. **Hepatitis** (inflammation of the liver)
2. **Gallstones** (cholelithiasis)
3. **Pancreatitis** (inflammation of the pancreas)
4. **Liver abscess**
5. **Peptic ulcer disease**

It is important to consult a healthcare professional for an accurate diagnosis and appropriate treatment. Diagnostic tests such as blood tests, ultrasound, or CT scans may be required to determine the exact cause of the sy